In [ ]:
# !pip install langchain langchain_groq langchain_core langchain_community langchain_google_genai python-dotenv pypdf faiss-gpu faiss-cpu

In [7]:
# import streamlit as st
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [9]:
llm=ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [10]:
prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)


In [13]:
def vector_embedding():
        embeddings=GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
        loader=PyPDFDirectoryLoader("./data") ## Data Ingestion
        docs=loader.load() ## Document Loading
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200) ## Chunk Creation
        final_documents=text_splitter.split_documents(docs[:20]) #splitting
        vectors=FAISS.from_documents(final_documents,embeddings) #vector OpenAI embeddings
        return vectors


In [14]:
vectors = vector_embedding()

In [15]:
question = input("Ask a question : ")

import time
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vectors.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
start=time.process_time()
response=retrieval_chain.invoke({'input':question})
print("Response time : ",time.process_time()-start)
print(response['answer'])
